In [1]:
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
import torch
from trl import SFTTrainer, setup_chat_format



c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Importing the dataset
dataset = load_dataset("yunfan-y/trump-qa", split="train")

# randomly select 500 rows
dataset = dataset.shuffle(seed=42).select(range(500))

# Load the tokenizer and model
model_name = "google/gemma-2-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
)

# Assuming setup_chat_format is a custom function you've defined elsewhere
# If not, you might need to implement or import it
model = get_peft_model(model, peft_config)

def format_chat_template(row):    
    row_json = [{"role": "user", "content": row["input"]},
               {"role": "assistant", "content": row["output"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Apply the formatting to the dataset
dataset = dataset.map(
    format_chat_template
)

print(dataset)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.17it/s]


Dataset({
    features: ['output', 'input', 'instruction', '__index_level_0__', 'text'],
    num_rows: 500
})


In [3]:
# Setting Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./result",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    group_by_length=True,
    report_to="wandb",
    weight_decay=0.01,
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

model.config.use_cache = False
trainer.train()

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 500/500 [00:00<00:00, 11642.69 examples/s]
c:\User

  0%|          | 1/250 [00:00<02:24,  1.72it/s]

{'loss': 5.4586, 'grad_norm': 4.815914154052734, 'learning_rate': 2e-05, 'epoch': 0.0}


  1%|          | 2/250 [00:00<01:30,  2.74it/s]

{'loss': 5.6908, 'grad_norm': 5.110575199127197, 'learning_rate': 4e-05, 'epoch': 0.01}


...

100%|██████████| 250/250 [00:52<00:00,  4.98it/s]

{'loss': 0.7469, 'grad_norm': 2.9067838191986084, 'learning_rate': 0.0, 'epoch': 1.0}


100%|██████████| 250/250 [00:52<00:00,  4.74it/s]

{'train_runtime': 53.7598, 'train_samples_per_second': 9.301, 'train_steps_per_second': 4.65, 'train_loss': 2.5909592967033386, 'epoch': 1.0}


TrainOutput(global_step=250, training_loss=2.5909592967033386, metrics={'train_runtime': 53.7598, 'train_samples_per_second': 9.301, 'train_steps_per_second': 4.65, 'total_flos': 268035239950848.0, 'train_loss': 2.5909592967033386, 'epoch': 1.0})